# Introduction
Shallow water gravity waves are horizontally propagating oscillations produced by large-scale disturbances and they can only exist in the presence of a free surface or a density discontinuity (thermocline). In this case study, the fluid has a free surface that experiences upward and downward perturbations. The perturbations create a sinusoidal pattern moving at some velocity, *u*.

We assume that the two fluids are incrompressible in order to exclude sound waves and isolate gravity waves.

The perturbation forms below are useful for this situation.

$u=\bar{u}+u^{\prime}, \quad h=H+h^{\prime}$

H is the mean depth of the lower layer and \bar{u} is a constant basic state zonal velocity. 

We are solving equation 7.22 from Holton's *An Introduction to Dynamic Meteorology* (2004): 

$$
\left(\frac{\partial}{\partial t}+\bar{u} \frac{\partial}{\partial x}\right)^2 h^{\prime}-\frac{g H \delta \rho}{\rho_1} \frac{\partial^2 h^{\prime}}{\partial x^2}=0 
$$
Expansion yields can be used to cancel terms in the first parenthesis: 
$$
\begin{aligned}


& \left(\frac{\partial^2}{\partial t^2}+u^2 \frac{\partial^2}{\partial x^2}\right) h^{\prime}-\frac{g H \delta p}{\rho_1} \frac{\partial^2 h^{\prime}}{\partial x^2} \\
& \frac{\partial^2 h^1}{\partial t^2}+u^2 \frac{\partial^2 h^{\prime}}{\partial x^2}-\frac{g+1 \delta \rho}{\rho_1} \frac{\partial^2 h^{\prime}}{\partial x^2} \\
& \frac{\partial^2 h^{\prime}}{\partial t^2}+\left(u^2-\frac{g+1 \delta p}{\rho_1}\right) \frac{\partial^2 h^{\prime}}{\partial x^2}=0 \\
& h^{\prime} \equiv h
\end{aligned}
$$
phase speed, $c: \quad c=u \pm \sqrt{g H}$  

Upper and lower layers are water and air. So, $\partial \rho_1 \approx \rho_1 $

$\sqrt{\mathrm{g} H} \approx 200 \mathrm{~m} / \mathrm{s}$ when $H=4 \mathrm{~km}$ as produced by large-scale disturbances, not wind

 $\frac{\partial^2 h}{\partial t^2}+c \frac{\partial^2 h}{\partial x^2}=0$

We will use this derivation to produce a simple two-dimensional model that has a flux boundary condition and a free surface. 




$$ \begin{align} u^{n+1}_i &= -u^{n-1}_i + 2u^n_i + 2C^2 \left(u^{n}_{i+1}-u^{n}_{i}\right),\quad i=0\\ u^{n+1}_i &= -u^{n-1}_i + 2u^n_i + 2C^2 \left(u^{n}_{i-1}-u^{n}_{i}\right),\quad i=N_x \end{align} $$


## Equations
# 

## Numerical Methods

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
import matplotlib
import matplotlib.animation as animation
matplotlib.use("TkAgg")

In [10]:
# parameters

xf = 24900*1000 #(m) circumference of earth at equator
xt = xf/2 #where you want the second wave speed to start
tf = 7*24*3600 #(s) 7 days in seconds
gH = 200 #(m/s) zonal wave speed in shallow water (4km)

# grid
dx = 24900
dt = 60
#dt = C*dx/c 
x = np.arange(0,xf,dx)
t = np.arange(0,tf,dt)

nx = len(x)
nt = len(t)

uv1 = 10 #average zonal velocity

# uv1 = 10 #average zonal velocity
# uv2 = -10 #average zonal velocity

# wave speed
c1 = (uv1 + gH)
c2 = (uv1 - gH)

# changing wave speed
lam = np.zeros(nx) #this is the lambda matrix
lam[x<=xt] = c1*dt/dx #sets the first half of space to have wave speed 1
lam[x>xt] = c2*dt/dx #sets the second half of space to have wave speed 2

# initial condition 
u1 = np.nan*np.ones([nx,nt])
u1[:,0] = np.exp(-(x[:]-0)**2 / 2) ## we have to initialize both 0th and 1th because of
u1[:,1] = np.exp(-(x[:]-0)**2 / 2) ## the presence of k-1 in the numerical equation. implies du/dt at t=0, =0

# matrices
data1 = np.array([(lam**2)*np.ones(nx), 2*(1-lam**2)*np.ones(nx), (lam**2)*np.ones(nx)])
diags1 = np.array([-1, 0, 1])
M = spdiags(data1, diags1, nx, nx).toarray()

#Reflective BC
M[0,0] = 2-lam[0]**2
M[nx-1,nx-1] = 2-lam[nx-1]**2

#Mirror BC
# M1[nx-1, nx-2] = 0
# M1[nx-1,nx-1] = 1

#Flux BC

In [11]:
# solve
for i in range(1,nt-1):
    u1[:,i+1] = M @ u1[:,i] - u1[:,i-1]

## Results

## Figures

In [12]:
fig, ax = plt.subplots(1,figsize = (8,5), dpi = 200)
## plotting u at all space at different times
ax.plot(x, u1[:,0])
ax.plot(x, u1[:,10])
ax.set_xlabel('x'); ax.set_ylabel('u');

## Video Walkthrough
(insert video link)

## Reference
Holton, J. R. (2004). An introduction to dynamic meteorology. In International Geophysics (Vol. 88). https://doi.org/10.1016/s0074-6142(08)x6005-x

In [13]:
# BEWARE RUNNING THIS IS CPU INTENSIVE 
fig, ax = plt.subplots()


line, = ax.plot(x, u1[:,0]) #ADJUST THIS LINE
ax.set_ylim([np.min(u1), np.max(u1)])

def animate(i):
    line.set_ydata(u1[:, i]) 
    return line,


ani = animation.FuncAnimation(
    fig, animate,frames=u1.shape[1], interval=0.0000000001) #interval is in ms based on frames saved

plt.show()
